In [15]:
using BenchmarkTools

# 文字列の分割（split）

In [6]:
line = "0"
for i in 1:10000
    line = line * ",$i"
end

In [9]:
@benchmark strs = split(line, ",")

BenchmarkTools.Trial: 
  memory estimate:  569.17 KiB
  allocs estimate:  10015
  --------------
  minimum time:     861.500 μs (0.00% GC)
  median time:      956.799 μs (0.00% GC)
  mean time:        1.178 ms (4.53% GC)
  maximum time:     32.675 ms (93.38% GC)
  --------------
  samples:          4225
  evals/sample:     1

# 配列の初期化

## push!で値を追加

### サイズ指定なし・型指定なし

In [17]:
@benchmark begin
    list = []
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  3.52 MiB
  allocs estimate:  99506
  --------------
  minimum time:     2.035 ms (0.00% GC)
  median time:      2.238 ms (0.00% GC)
  mean time:        2.790 ms (9.78% GC)
  maximum time:     20.212 ms (0.00% GC)
  --------------
  samples:          1788
  evals/sample:     1

### サイズ指定なし・型指定あり

In [16]:
@benchmark begin
    list = Vector{Int64}()
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  2.00 MiB
  allocs estimate:  17
  --------------
  minimum time:     949.800 μs (0.00% GC)
  median time:      1.005 ms (0.00% GC)
  mean time:        1.172 ms (0.00% GC)
  maximum time:     9.379 ms (0.00% GC)
  --------------
  samples:          3797
  evals/sample:     1

### サイズヒントあり・型指定あり

In [19]:
@benchmark begin
    list = Vector{Int64}()
    sizehint!(list, 100000)
    for i in 1:100000
        push!(list, i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  781.39 KiB
  allocs estimate:  2
  --------------
  minimum time:     558.200 μs (0.00% GC)
  median time:      848.000 μs (0.00% GC)
  mean time:        920.887 μs (0.00% GC)
  maximum time:     10.502 ms (0.00% GC)
  --------------
  samples:          5028
  evals/sample:     1

## list[index] = value で値をセット

In [20]:
@benchmark begin
    list = Vector{Int64}(undef, 100000)
    for i in 1:100000
        list[i] = i
    end
end

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     75.400 μs (0.00% GC)
  median time:      94.201 μs (0.00% GC)
  mean time:        237.837 μs (13.86% GC)
  maximum time:     5.976 ms (56.88% GC)
  --------------
  samples:          10000
  evals/sample:     1

**push!でなく値をセットする方式が10倍速い（push!による初期化は遅い）**

## リスト内包表記による初期化

### 型指定なし

In [23]:
@benchmark list = [i for i in 1:100000]

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     42.201 μs (0.00% GC)
  median time:      87.600 μs (0.00% GC)
  mean time:        131.865 μs (9.79% GC)
  maximum time:     7.840 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

**リストの初期化はリスト内包表記が速い**

### 型指定あり

In [24]:
@benchmark list = Int64[i for i in 1:100000]

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     60.499 μs (0.00% GC)
  median time:      88.401 μs (0.00% GC)
  mean time:        116.101 μs (9.37% GC)
  maximum time:     7.253 ms (98.38% GC)
  --------------
  samples:          10000
  evals/sample:     1

## リストの初期化まとめ

- リスト内包表記またはサイズ指定して値をセットする方式が速い
- リスト内包表記の型指定は速度に影響しないかも（今回試したのがプリミティブだからかも）
- push!は遅いが便利なのでパフォーマンスを気にしない場面で用いる

# Dictの初期化

## push!による初期化

In [10]:
@benchmark begin
    dict = Dict{Int64,Int64}()
    for i in 1:10000
        push!(dict, i => i)
    end
end

BenchmarkTools.Trial: 
  memory estimate:  364.61 KiB
  allocs estimate:  24
  --------------
  minimum time:     275.700 μs (0.00% GC)
  median time:      405.000 μs (0.00% GC)
  mean time:        452.312 μs (6.36% GC)
  maximum time:     9.258 ms (90.40% GC)
  --------------
  samples:          10000
  evals/sample:     1

## 辞書内包表記による初期化

In [13]:
@benchmark begin
    dict = Dict{Int64,Int64}(
        i => i for i in 1:10000
    )
end

BenchmarkTools.Trial: 
  memory estimate:  364.61 KiB
  allocs estimate:  24
  --------------
  minimum time:     276.900 μs (0.00% GC)
  median time:      405.300 μs (0.00% GC)
  mean time:        451.900 μs (6.20% GC)
  maximum time:     12.583 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## リストを作って初期化

### Dict(zip(keys, values))

In [27]:
@benchmark begin
    keys = Int64[i for i in 1:10000]
    values = Int64[i for i in 1:10000]
    dict = Dict(zip(keys,values))
end

BenchmarkTools.Trial: 
  memory estimate:  521.06 KiB
  allocs estimate:  30
  --------------
  minimum time:     362.300 μs (0.00% GC)
  median time:      408.750 μs (0.00% GC)
  mean time:        540.951 μs (4.64% GC)
  maximum time:     9.668 ms (90.15% GC)
  --------------
  samples:          9180
  evals/sample:     1

### Dict(keys .=> values)

In [29]:
@benchmark begin
    keys = Int64[i for i in 1:10000]
    values = Int64[i for i in 1:10000]
    dict = Dict(keys .=> values)
end

BenchmarkTools.Trial: 
  memory estimate:  677.34 KiB
  allocs estimate:  30
  --------------
  minimum time:     354.500 μs (0.00% GC)
  median time:      398.901 μs (0.00% GC)
  mean time:        550.500 μs (5.67% GC)
  maximum time:     9.265 ms (89.96% GC)
  --------------
  samples:          9010
  evals/sample:     1

## Dictの初期化まとめ 

- 初期化方法で速度に差はない！